Lab03
=====

### \* 선형 회귀 분석

단순 회귀 분석(독립변수의 개수: 1)

$Y = B_0 + B_1X_1$

다중 회귀 분석(독립 변수의 개수: 2개 이상)

$Y = B_0 + B_1X_1 + B_2X_2+ \cdot\cdot\cdot B_dX_d$
    
1. 모델이 적합한가
    - $R^2$ (결정계수):
      <br>  종속 변수의 총 변화량 중 모델이 잡아낼 수 있는 변화량의 비율
![R2](./Images/R2.png)

    (다중 회귀 분석에서는 독립변수의 수가 증가하면 함께 증가하는 $R^2$의 특징을 보완하는 수정된 $R^2$를 사용합니다.)
    

2. 선형 회귀 모델이 의미 있는가(회귀계수가 독립변수의 영향력을 잘 표현하는가)
    - 독립변수들간 서로 독립적이어야 한다.(다중 공선성 문제가 없어야 합니다.)
    
    ex) Y = 2(X1) + 3(X2) <다중 공선성 문제가 있는 X1, X2>

    => X1이 설명력 있는 독립 변수라면 p-value가 유의 수준보다 작아 유의한 통계량이 됩니다.

    (문제 발생)그런데, X2가 설명할 부분을 X1이 가져가 버렸기 때문에 X2의 설명력은 작아지게 됩니다.

    #### \* 이를 방지하는 방법
        ___\- 변수 선택법(Feature Selection)으로 의존적인 변수 삭제하는 방법___
        <br>\- PCA(principal component analysis) 방법으로 의존적인 성분 삭제하는 방법
        <br>\- 정규화(regularized) 방법

#### 3. 변수 선택법(Feature Selection)으로 의존적인 변수 삭제하는 방법

### < Stepwise selection >

### p-value 분석을 통한 변수 선택(Feature Selection)
+ 가설 검정을 통해서 판단합니다.
    + 귀무가설을 회귀 계수 = 0이라고 설정(회귀 계수가 유의미 하지 않다.)
    + 그 회귀 계수가 유의미하다고 판단하면 기각


+ 다중공선성이 생기면 해당 변수의 설명력은 약해집니다.
+ 이는 변수의 표준오차의 증가로 드러납니다.
+ 검정 통계량 = (추정된 회귀 계수 - H0) / 그 계수의 추정 표준 오차

        1) 여기서 표준 오차는 표본의 실제관측치가 표본 회귀선상의 추정치에서 얼마나 흩어져 있나를 측정한 값입니다.

        2) 표준오차가 작으면 작을수록 실제값과 추정값간의 차이가 없다고 볼수 있습니다.

        3) 표준오차 구하기 
<center>$s = \sqrt{\sum \frac{(Y_i-\hat{Y})^2}{n-2}}$

        (추정표준오차의 제곱은 잔차의 분산입니다.)


+ 이 검정 통계량의 절대값이 클수록 p-value는 작아져서(표준 오차가 커졌기 때문) 귀무가설을 기각할 수 있게 됩니다.





![stepwise2](./Images/stepwise_2.png)

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in= 0.05, 
                       threshold_out = 0.01, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

## 예제
__다중 회귀분석시 다중 공선성 문제 방지 방법__

In [ ]:
from os.path import join
import numpy as np

from sklearn import linear_model

from sklearn.metrics import r2_score

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

import statsmodels.api as sm

import pandas as pd
from pandas.plotting import scatter_matrix

In [ ]:
df = pd.read_csv(join('data','diabetesDataset.csv'))
print(df.shape)

In [ ]:
for dim in df.columns:
        df[dim] -= np.min(df[dim])
        df[dim] /= np.max(df[dim])

In [ ]:
df.hist(bins = 20, figsize=(10, 15))
plt.show()

## 1) 상관관계분석

In [ ]:
df.corr()

## 2) Stepwise selection

    통계적으로 유의미한 변수를 선택합니다.

In [ ]:
dfx = df[df.columns[:-1]]
dfy = df[['y']]

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in= 0.05, 
                       threshold_out = 0.01, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(dfx, dfy)

print('resulting features:')
print(result)

newlabel = result

In [ ]:
diabetes_X_multi = df[newlabel].values.tolist()
diabetes_Y_multi = df[['y']].values.tolist()

# 선형회귀 추정기 생성
lr = linear_model.LinearRegression() 

# input 대해 선형 회귀(모델 파라미터 추정)
lr.fit(diabetes_X_multi, diabetes_Y_multi)

# 회귀식으로 데이터의 결과 추정
diabetes_y_pred_multi = lr.predict(diabetes_X_multi)

# R2 scroe
R2 = r2_score(diabetes_Y_multi, diabetes_y_pred_multi)
print('R2 score: %.2f' % R2)

# adj R2 score
n = len(diabetes_X_multi)
p = len(newlabel)

Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
print('Adj R2 score: %.2f' % Adj_r2)

In [ ]:
import itertools

newlabel = ['age', 'sex', 'bmi', 'map', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']
combs = []
adj_list = []
label_list = []
for i in range(1, len(newlabel)+1):
    els = [list(x) for x in itertools.combinations(newlabel, i)]
    combs.extend(els) # list 원소만을 append(일반적인 append는 list 자체를 추가한다.)
for i in range(len(combs)):
    print('label', combs[i])
    label_list.append(combs[i])
    dfx = df[combs[i]].values.tolist()
    dfy = df[['y']].values.tolist()
    print(np.shape(dfx))

    # 선형회귀 추정기 생성
    lr2 = linear_model.LinearRegression()

    # input 대해 선형 회귀(모델 파라미터 추정)
    lr2.fit(dfx, dfy)

    # 모델 파라미터 출력
    print('Model parameters: \n')
    rgstr = ''
    for j in range(len(lr2.coef_[0])):
        print("b%d" %(len(lr2.coef_[0])-j),": ", "%f" %lr2.coef_[0,j])
        rgstr = ' + '+repr(lr2.coef_[0,j]) + '*x'+repr(len(lr2.coef_[0])-j) + rgstr

    print('b0:', lr2.intercept_[0])
    print()
    rgstr = repr(lr2.intercept_[0]) + rgstr
    print('y = ', rgstr)

    y_pred = lr2.predict(dfx)
    n = len(dfx)
    p = len(combs[i])

    R2 = r2_score(dfy, y_pred)
    Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
    adj_list.append(Adj_r2)
    print('Adj R2 score: %.2f' %  Adj_r2)
    print('**************************************************************')

In [ ]:
print('Max Adj R2 score: %.2f' %  np.max(adj_list))
print('Selected label :', label_list[np.argmax(adj_list)])

# 회귀 프로젝트
배운 내용을 바탕으로 회귀 모델을 만들고 분석해 보세요.

# 1. 데이터 살펴보기

이번 시간 사용할 데이터는 Combined Cycle Power Plant Data Set입니다. 이 데이터 세트는 발전소가 최대 부하로 작동하도록 설정된 동안 수집된 데이터들입니다.
5개의 컬럼을 가지고 있으며, 각 컬럼들은 시간별 평균 주변 변수로 구성됩니다.
구성 값들은 매 초마다 주변 변수를 기록하는 공장 주변에 위치한 다양한 센서의 측정 값을 평균한 값입니다.

### Feature Description
- AT : 주변 온도(범위 1.81 - 37.11(°C))
- V  : 배기 진공 압력()
- AP : 주변 압력(범위 992.89 - 1033.30(milibar))
- RH : 상대 습도(범위 25.56 - 100.16(%))
- PE : 공장의 시간당 전기 에너지 출력(범위 420.26 - 495.76(MW))

In [ ]:
df = pd.read_csv(join('data','Folds5x2_pp_1.csv'))
print(df.shape)

In [ ]:
df.head()

# 2.간단한 전처리

In [ ]:
for dim in df.columns:
        df[dim] -= np.min(df[dim])
        df[dim] /= np.max(df[dim])

독립변수와 종속변수를 분리합니다.

In [ ]:
dfx = df[df.columns[:-1]]
dfy = df[['PE']]

In [ ]:
dfx.head(2)

In [ ]:
dfy.head(2)

# 2.1 단순 선형 회귀 분석

In [ ]:
import itertools

newlabel = df.columns[:-1]
combs = []

adj_list = []
label_list = []


els = [list(x) for x in itertools.combinations(newlabel, 1)]
combs.extend(els) # list 원소만을 append(일반적인 append는 list 자체를 추가한다.)
    
for i in range(len(combs)):
    print('label', combs[i])
    label_list.append(combs[i])
    dfx_tmp = dfx[combs[i]].values.tolist()
    dfy_tmp = dfy.values.tolist()
    print(np.shape(dfx_tmp))

    # 선형회귀 추정기 생성
    lr2 = linear_model.LinearRegression()

    # input 대해 선형 회귀(모델 파라미터 추정)
    lr2.fit(dfx_tmp, dfy_tmp)

    # 모델 파라미터 출력
    print('Model parameters: \n')
    rgstr = ''
    for j in range(len(lr2.coef_[0])):
        print("b%d" %(len(lr2.coef_[0])-j),": ", "%f" %lr2.coef_[0,j])
        rgstr = ' + '+repr(lr2.coef_[0,j]) + '*x'+repr(len(lr2.coef_[0])-j) + rgstr

    print('b0:', lr2.intercept_[0])
    print()
    rgstr = repr(lr2.intercept_[0]) + rgstr
    print('y = ', rgstr)

    y_pred = lr2.predict(dfx_tmp)
    n = len(dfx_tmp)
    p = len(combs[i])

    R2 = r2_score(dfy_tmp, y_pred)
    Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
    adj_list.append(Adj_r2)
    print('Adj R2 score: %.2f' %  Adj_r2)
    
    # Plot

    # 데이터 좌표 plot
    plt.scatter(dfx_tmp, dfy_tmp,  color='black', label='Data Point(patients)')

    # 회귀 결과 plot
    plt.plot(dfx_tmp, y_pred, color='blue', linewidth=3, label='Linear Regression')

    # x축 label
    plt.xlabel(combs[i])

    #y축 label
    plt.ylabel('PE')
    plt.legend()
    plt.show()
    print('**************************************************************')

# 2.2 다중 선형 회귀 분석 

## 2.2.1 상관관계 분석

In [ ]:
df.corr()

In [ ]:
scatter_matrix(df, figsize = (20,15))
plt.show()

- 종속변수인 PE와 다른 변수들간에 상관관계가 있다는 것을 확인할 수 있습니다.

- V와 AT 변수간의 상관관계가 있음을 확인 할 수 있습니다.

## 2.2.2 다중공선성 방지

### Stepwise selection

    통계적으로 유의미한 변수를 선택합니다.

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in= 0.05, 
                       threshold_out = 0.01, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(dfx, dfy)

print('resulting features:')
print(result)

newlabel = result

###  회귀식 추정 및 평가

In [ ]:
dfx_multi = df[newlabel]
dfy_multi = df[['PE']]

# 선형회귀 추정기 생성
lr = linear_model.LinearRegression() 

# input 대해 선형 회귀(모델 파라미터 추정)
lr.fit(dfx_multi.values.tolist(), dfy_multi.values.tolist())

# 회귀식으로 데이터의 결과 추정
dfy_y_pred_multi = lr.predict(dfx_multi.values.tolist())

# R2 scroe
R2 = r2_score(dfy_multi.values.tolist(), dfy_y_pred_multi)
print('R2 score: %.2f' % R2)

# adj R2 score
n = len(dfx_multi)
p = len(newlabel)

Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
print('Adj R2 score: %.2f' % Adj_r2)